# Diplomacy LLM Framework Notebook

This Jupyter notebook provides a self-contained implementation of the Diplomacy LLM Agent Framework. It allows you to run a full Diplomacy simulation with LLM-powered agents directly within this environment.

The notebook is structured sequentially:
1.  **Setup**: Importing necessary libraries and configuring the essential API key.
2.  **Component Definitions**: Defining the core data structures, base agent class, LLM agent logic, game engine adapter, and the main orchestrator.
3.  **Configuration**: Setting parameters for the LLM agent.
4.  **Instantiation**: Creating instances of the agents and the orchestrator.
5.  **Execution**: Running the game simulation.

**Important Note:** Before running the cells, ensure you have set up the correct Conda environment (`diplomacy_env`) as described in the `README.md` and selected it as the kernel for this notebook.

---

### Cell 1: Imports and API Key Setup

The first code cell below handles crucial setup steps:
*   Imports standard Python libraries (`os`, `datetime`, `json`, etc.) and necessary third-party packages (`diplomacy`, `openai`).
*   **Crucially, it defines the `OPENROUTER_API_KEY` variable.** You **MUST** replace the placeholder value with your actual OpenRouter API key for the LLM agents to function. It also includes basic verification to check if the key has been set.

---

In [ ]:
# Cell 1: Basic Imports and API Key

import sys
import os
import datetime
import json
import re
import time
from typing import List, Dict, Any, Type, Optional, NamedTuple, OrderedDict # Import necessary typing components
from collections import OrderedDict # Needed for memory in orchestrator

# --- Define API Key ---
# !!! IMPORTANT: please replace the placeholder with your actual OpenRouter API key !!!
OPENROUTER_API_KEY = "YOUR_KEY_HERE"

# Verification (Optional but recommended)
if not OPENROUTER_API_KEY or OPENROUTER_API_KEY == "sk-or-v1-YOUR_KEY_HERE":
    print("ERROR: Please replace 'sk-or-v1-YOUR_KEY_HERE' with your actual OpenRouter API key in the line above.")
    api_key_defined = False
else:
    print("OPENROUTER_API_KEY variable is set.")
    api_key_defined = True

# Attempt to import diplomacy library (needed for EngineAdapter later)
try:
    from diplomacy import Game
    from diplomacy.utils.export import to_saved_game_format # If we want saving later
    print("Diplomacy library imported successfully.")
except ImportError:
    print("ERROR: diplomacy library not found. Please ensure it's installed in the 'diplomacy_env' environment (`pip install diplomacy`).")
    # Define dummy classes to potentially avoid immediate errors later, but functionality will be broken
    class Game: pass
    def to_saved_game_format(game): return {}

# Attempt to import OpenAI library (needed for LLMAgent later)
try:
    from openai import OpenAI, APITimeoutError, APIConnectionError, RateLimitError, APIStatusError
    print("OpenAI library imported successfully.")
except ImportError:
    print("ERROR: openai package not found. Please install it (`pip install openai`)")
    class OpenAI: pass
    class APITimeoutError(Exception): pass
    class APIConnectionError(Exception): pass
    class RateLimitError(Exception): pass
    class APIStatusError(Exception): pass


---
## Framework Component Definitions

The following cells define the core components of the LLM agent framework directly within this notebook for self-containment. This includes data structures, the base agent class, the engine adapter, the LLM agent, and the main orchestrator.

### Data Structures

To ensure clear communication and data handling between different parts of the framework (like the orchestrator, agents, and engine adapter), we define several custom data structures using Python's `NamedTuple`. These structures provide a standardized way to represent game state, messages, agent context, and agent actions. The next cell defines these core data types.

---


In [ ]:
# Cell 3: Data Structures Definition

# Define type aliases for clarity
PowerName = str
PhaseName = str
TerritoryName = str

class Message(NamedTuple):
    """Represents a message sent between powers."""
    sender: PowerName
    recipient: PowerName
    phase: PhaseName
    content: str
    timestamp: Optional[datetime.datetime] = None # Allow optional timestamp

class PublicGameState(NamedTuple):
    """Holds the public information about the current game state."""
    centers: Dict[PowerName, List[TerritoryName]]
    units: Dict[PowerName, List[List[str]]] # e.g., {'FRANCE': [['A', 'PAR'], ['F', 'BRE']]}
    builds_disbands: Dict[PowerName, int] # e.g., {'FRANCE': 1, 'GERMANY': -1}

class PhaseInfo(NamedTuple):
    """Information about the current phase and interaction step."""
    phase_name: PhaseName # e.g., "S1901M", "F1901M", "W1901A"
    interaction_type: str # e.g., "NEGOTIATION_ROUND_1", "ORDER_SUBMISSION"

class HistorySummary(NamedTuple):
    """Holds the narrative summary of the game history."""
    summary_text: str

class AgentContextBundle(NamedTuple):
    """Bundles all information provided to an agent for its turn."""
    power_name: PowerName
    public_game_state: PublicGameState
    current_phase_info: PhaseInfo
    communication_inbox: List[Message]
    private_memory_snippet: Dict[str, Any] # Using simple dict for memory snippet
    agent_instructions: str
    history_summary: HistorySummary
    previous_phase_results: Optional[Dict[PowerName, List[str]]] # Order results from last phase

class ActionToolCall(NamedTuple):
    """Represents an action requested by an agent."""
    tool_name: str
    arguments: Dict[str, Any] = {} # Default to empty dict

print("Framework data structures defined.")

---

### Base Agent Class

To allow for different types of agents (e.g., rule-based, LLM-based, human input) to interact with the framework consistently, we define an abstract base class called `BaseAgent`. This class establishes the fundamental interface that all agents must implement. The most important part of this interface is the `take_turn` method, which the orchestrator will call to get the agent's actions for the current step.

---

In [ ]:
# Cell 4: Base Agent Definition

from abc import ABC, abstractmethod

class BaseAgent(ABC):
    """Abstract base class for all Diplomacy agents."""

    def __init__(self, power_name: PowerName):
        """
        Initializes the agent.

        Args:
            power_name: The name of the power this agent controls (e.g., 'FRANCE').
        """
        if not power_name or not isinstance(power_name, str):
             raise ValueError("Agent must be initialized with a valid power name.")
        self.power_name = power_name
        print(f"BaseAgent: Initialized agent for {self.power_name}")

    @abstractmethod
    def take_turn(self, context_bundle: AgentContextBundle, run_log_dir: Optional[str] = None) -> List[ActionToolCall]:
        """
        The core method called by the framework for the agent to decide its actions.

        Args:
            context_bundle: An object containing all relevant information for the current decision context.
            run_log_dir: Optional path to a directory for agent-specific logging.

        Returns:
            A list of ActionToolCall objects representing the agent's desired actions for this turn/step.
            The list should end with a 'finish_*' action (e.g., finish_negotiation_round, finish_orders).
        """
        pass

    # Optional: Add a static method for summarization if needed, though LLMAgent handles it now
    # @staticmethod
    # def summarize_history(...) -> str:
    #    pass

print("BaseAgent class defined.")

---

### LLM Agent Implementation

This cell defines the `LLMAgent`, a concrete implementation of the `BaseAgent`. This agent uses a Large Language Model (LLM) accessed via an OpenAI-compatible API (specifically configured for OpenRouter in this notebook) to make decisions.

Key responsibilities of this class include:
*   **Prompt Formatting (`_format_prompt`)**: Dynamically creating detailed prompts based on the current game state, messages, memory, and phase instructions.
*   **LLM Interaction (`take_turn`)**: Calling the LLM API with the formatted prompt, handling potential errors and retries.
*   **Response Parsing (`_parse_llm_response`)**: Cleaning and parsing the LLM's JSON response into a list of `ActionToolCall` objects that the framework can understand.
*   **History Summarization (`summarize_history`)**: A static method used by the orchestrator to generate narrative summaries of completed game phases using the LLM.

---


In [ ]:
#Cell 5: LLM Agent Definition

# Ensure BaseAgent is defined (Cell 4)
# Ensure data structures defined (Cell 3)
# Ensure API Key variable defined (Cell 1)
# Ensure OpenAI library loaded (Cell 1)

class LLMAgent(BaseAgent):
    """An agent that uses an LLM via the OpenAI client interface to decide actions."""

    def __init__(self, power_name: str, llm_model_name: str, llm_temperature: float, llm_max_tokens: int):
        super().__init__(power_name)
        # Use OpenRouter configuration - Use API key variable from notebook scope
        self.api_key = OPENROUTER_API_KEY # Use variable directly
        self.base_url = "https://openrouter.ai/api/v1"
        if not self.api_key or self.api_key == "sk-or-v1-YOUR_KEY_HERE": # Add check for placeholder
            raise ValueError("OpenRouter API key not found or not set in the notebook variable OPENROUTER_API_KEY.")

        # Ensure OpenAI class is available before trying to instantiate
        if 'OpenAI' in globals() and hasattr(OpenAI, '__call__'):
            try:
                 self.client = OpenAI(
                    base_url=self.base_url,
                    api_key=self.api_key,
                    timeout=60.0 # Add a reasonable timeout
                 )
            except Exception as e:
                 print(f"ERROR: Failed to initialize OpenAI client: {e}")
                 self.client = None # Set client to None if init fails
        else:
             print("ERROR: OpenAI client library not loaded correctly.")
             self.client = None

        # Store original parameters for reference and potential use in summarization
        self.llm_model_name = llm_model_name
        self.llm_temperature = llm_temperature
        self.llm_max_tokens = llm_max_tokens

        # Parameters specifically for the API call (can differ from stored ones if needed)
        self.model_name = llm_model_name # Use the passed model name for API calls
        self.max_tokens = llm_max_tokens
        self.temperature = llm_temperature
        self.memory: Dict[str, Any] = {} # Initialize agent memory (kept simple)
        self.log_filename: Optional[str] = None # Use Optional typing


    def _format_prompt(self, context_bundle: AgentContextBundle) -> str:
        """Formats the agent context into a string prompt for the LLM."""
        phase_info = context_bundle.current_phase_info
        interaction_type = phase_info.interaction_type

        prompt_lines = []
        prompt_lines.append(f"You are {context_bundle.power_name}, playing Diplomacy.")

        # --- Order Submission Focused Prompt ---
        if interaction_type == "ORDER_SUBMISSION":
            prompt_lines.append("\n== CURRENT GAME STATE (Focus!) ==")
            your_units_list = context_bundle.public_game_state.units.get(self.power_name, [])
            your_units_str_simple = ", ".join([f"{u[0]} {u[1]}" for u in your_units_list])
            all_units_str = json.dumps(context_bundle.public_game_state.units)
            centers_str = json.dumps(context_bundle.public_game_state.centers)
            prompt_lines.append(f"  Your Units: {your_units_str_simple}")
            prompt_lines.append(f"  All Units: {all_units_str}")
            prompt_lines.append(f"  Supply Centers: {centers_str}")
            if context_bundle.public_game_state.builds_disbands:
                 builds_str = json.dumps(context_bundle.public_game_state.builds_disbands)
                 prompt_lines.append(f"  Builds/Disbands Available: {builds_str}")
            prompt_lines.append("")

            prompt_lines.append("== CURRENT SITUATION ==")
            prompt_lines.append(f"Phase: {context_bundle.current_phase_info.phase_name}")
            prompt_lines.append(f"Your Interaction Step: {interaction_type}")

            inbox = context_bundle.communication_inbox
            if inbox:
                prompt_lines.append("\n== MESSAGES RECEIVED THIS STEP ==")
                for msg in inbox:
                    prompt_lines.append(f"  From {msg.sender}: {msg.content}")
                prompt_lines.append("")
            memory = context_bundle.private_memory_snippet
            if memory:
                 prompt_lines.append("== YOUR PRIVATE NOTES (Memory) ==")
                 try:
                      memory_str = json.dumps(memory, indent=2)
                      prompt_lines.append(memory_str)
                 except TypeError:
                      prompt_lines.append(str(memory)) # Fallback for non-serializable memory
                 prompt_lines.append("")

            prompt_lines.append("== INSTRUCTIONS & AVAILABLE TOOLS FOR ORDER SUBMISSION ==")
            prompt_lines.append("Your response MUST be a valid JSON list containing allowed tool calls.")
            prompt_lines.append("Pay EXTREME attention to JSON syntax and required arguments.")
            phase_suffix = context_bundle.current_phase_info.phase_name[-1]
            prompt_lines.append("\nGoal: Submit your final, binding orders for this phase.")

            # Movement Phase Orders
            if phase_suffix == 'M':
                prompt_lines.append("\nPhase Type: MOVEMENT")
                prompt_lines.append("CRITICAL: Base your orders *only* on the 'Current Game State: Your Units' listed ABOVE.")
                prompt_lines.append("Ensure you issue exactly ONE MOVE, HOLD, or SUPPORT order for EACH of your current units.")
                prompt_lines.append("REQUIRED STEP: Use log_thought FIRST. In its 'thought' argument, provide your step-by-step reasoning: explicitly list your current units, evaluate options for each, explain your final decision, and state the planned single order for EACH unit.")
                prompt_lines.append("\nAllowed Tools for Order Submission (Movement):")
                prompt_lines.append("  - log_thought(thought: str) # Use FIRST. Include detailed reasoning and final plan for ALL units.")
                prompt_lines.append("  - submit_order(order_string: str)")
                prompt_lines.append("      # Format examples: 'A PAR H', 'F MAO - SPA', 'A MAR S A PAR', 'A PIC S F MAO - SPA'...")
            # Adjustment Phase Orders
            elif phase_suffix == 'A':
                prompt_lines.append("\nPhase Type: ADJUSTMENT")
                prompt_lines.append("CRITICAL: Issue BUILD or DISBAND orders based on 'Builds/Disbands Available', or WAIVE builds. Refer to 'Current Game State'.")
                prompt_lines.append("You MUST account for the exact number of builds (+N) or disbands (-N) required.")
                prompt_lines.append("**If Builds/Disbands count is 0, submit NO orders, just log thought and finish.**")
                prompt_lines.append("REQUIRED STEP: Use log_thought FIRST. In its 'thought' argument, provide reasoning: state your build/disband count, analyze options, and list the specific BUILD/DISBAND/WAIVE orders (or state 'No action needed').")
                prompt_lines.append("\nAllowed Tools for Order Submission (Adjustment):")
                prompt_lines.append("  - log_thought(thought: str) # Use FIRST. Include reasoning and final plan for build/disband/waive (or 'No action').")
                prompt_lines.append("  - submit_order(order_string: str)")
                prompt_lines.append("      # Format examples:")
                prompt_lines.append("      #   Build: 'A PAR B' or 'F BRE B' (MUST be an owned, vacant, HOME supply center)")
                prompt_lines.append("      #   Disband: 'A RUH D' or 'F KIE D'")
                prompt_lines.append("      #   Waive Build: 'WAIVE' (Submit one 'WAIVE' per unused build if builds > 0)")
            # Retreat Phase Orders
            else: # Fallback for Retreat ('R') or other unknown phases
                prompt_lines.append("\nPhase Type: RETREAT / OTHER")
                prompt_lines.append("CRITICAL: Submit orders appropriate for this phase type (e.g., Retreat or Disband for units needing retreats). Consult rules.")
                prompt_lines.append("REQUIRED STEP: Use log_thought FIRST. In its 'thought' argument, provide reasoning: list units needing orders, evaluate retreat/disband options, explain choice, and state the planned order for each.")
                prompt_lines.append("\nAllowed Tools for Order Submission (Other):")
                prompt_lines.append("  - log_thought(thought: str) # Use FIRST. Include reasoning and final plan for units needing retreat/disband orders.")
                prompt_lines.append("  - submit_order(order_string: str) # Use appropriate format (e.g., 'A MUN R BER', 'F NAP D').")

            prompt_lines.append("\nCommon Tools Available:")
            prompt_lines.append("  - update_memory(key: str, value: any)")
            prompt_lines.append("\nIMPORTANT: Your response list MUST end with the finish_orders tool call:")
            prompt_lines.append("  - finish_orders()")
            prompt_lines.append("\nExample Final JSON Structure (Movement):")
            prompt_lines.append("```json")
            prompt_lines.append("[")
            prompt_lines.append("  { \"tool_name\": \"log_thought\", \"arguments\": { \"thought\": \"Reasoning: My units are F MAO, A PIC, A MAR. England might move to ENG, so MAO to SPA is risky but necessary for Iberia. PIC support needed. MAR holds. Final Plan: F MAO -> SPA, A PIC S F MAO -> SPA, A MAR H.\" } },")
            prompt_lines.append("  { \"tool_name\": \"submit_order\", \"arguments\": { \"order_string\": \"F MAO - SPA\" } },")
            prompt_lines.append("  { \"tool_name\": \"submit_order\", \"arguments\": { \"order_string\": \"A PIC S F MAO - SPA\" } },")
            prompt_lines.append("  { \"tool_name\": \"submit_order\", \"arguments\": { \"order_string\": \"A MAR H\" } },")
            prompt_lines.append("  { \"tool_name\": \"finish_orders\", \"arguments\": {} }")
            prompt_lines.append("]")
            prompt_lines.append("```")

        # --- Negotiation Focused Prompt (Includes Summary) ---
        elif interaction_type.startswith("NEGOTIATION_ROUND"):
            prompt_lines.append("== OVERALL GOAL ==")
            prompt_lines.append(context_bundle.agent_instructions) # Assuming this is populated by orchestrator

            prompt_lines.append("\n== CURRENT GAME SUMMARY ==")
            prompt_lines.append(context_bundle.history_summary.summary_text if context_bundle.history_summary.summary_text else "The game has just begun.")

            prompt_lines.append("\n== CURRENT SITUATION ==")
            prompt_lines.append(f"Phase: {context_bundle.current_phase_info.phase_name}")
            prompt_lines.append(f"Your Interaction Step: {interaction_type}")
            prompt_lines.append("\n== CURRENT GAME STATE ==")
            your_units_list = context_bundle.public_game_state.units.get(self.power_name, [])
            your_units_str_simple = ", ".join([f"{u[0]} {u[1]}" for u in your_units_list])
            all_units_str = json.dumps(context_bundle.public_game_state.units)
            centers_str = json.dumps(context_bundle.public_game_state.centers)
            prompt_lines.append(f"  Your Units: {your_units_str_simple}")
            prompt_lines.append(f"  All Units: {all_units_str}")
            prompt_lines.append(f"  Supply Centers: {centers_str}")
            if context_bundle.public_game_state.builds_disbands:
                 builds_str = json.dumps(context_bundle.public_game_state.builds_disbands)
                 prompt_lines.append(f"  Builds/Disbands Available: {builds_str}")
            prompt_lines.append("")

            inbox = context_bundle.communication_inbox
            if inbox:
                prompt_lines.append("== MESSAGES RECEIVED THIS STEP ==")
                for msg in inbox:
                    prompt_lines.append(f"  From {msg.sender}: {msg.content}")
                prompt_lines.append("")
            else:
                 prompt_lines.append("== MESSAGES RECEIVED THIS STEP ==")
                 prompt_lines.append("  (None)")
                 prompt_lines.append("")
            memory = context_bundle.private_memory_snippet
            if memory:
                 prompt_lines.append("== YOUR PRIVATE NOTES (Memory) ==")
                 try:
                      memory_str = json.dumps(memory, indent=2)
                      prompt_lines.append(memory_str)
                 except TypeError:
                      prompt_lines.append(str(memory))
                 prompt_lines.append("")

            prompt_lines.append("== INSTRUCTIONS & AVAILABLE TOOLS FOR NEGOTIATION ==")
            prompt_lines.append("Your response MUST be a valid JSON list containing allowed tool calls.")
            prompt_lines.append("Pay EXTREME attention to JSON syntax and required arguments.")
            prompt_lines.append(f"\nGoal: Communicate with other powers, gather info, propose deals, update your memory. Current round: {interaction_type.split('_')[-1]}")
            prompt_lines.append("REQUIRED STEP: Use log_thought FIRST. In its 'thought' argument, provide your reasoning: analyze messages/state, decide communication strategy (who to talk to, what to say/ask), and summarize your planned actions for this round.")
            prompt_lines.append("\nAllowed Tools for Negotiation:")
            prompt_lines.append("  - log_thought(thought: str) # Use FIRST. Include detailed reasoning and plan for this round.")
            prompt_lines.append("  - send_message(recipient: PowerName, message_content: str)") # Simplified tool name/args
            prompt_lines.append("  - update_memory(key: str, value: any)")
            prompt_lines.append("\nIMPORTANT: Your response list MUST end with the finish_negotiation_round tool call:")
            prompt_lines.append("  - finish_negotiation_round()")
            prompt_lines.append("\nExample Final JSON Structure:")
            prompt_lines.append("```json")
            prompt_lines.append("[")
            prompt_lines.append("  { \"tool_name\": \"log_thought\", \"arguments\": { \"thought\": \"Analysis: France seems friendly based on message. Germany is quiet. Italy has units nearby. Plan: Propose alliance with France against Germany. Ask Italy about intentions. Remember France's offer.\" } },")
            prompt_lines.append("  { \"tool_name\": \"send_message\", \"arguments\": { \"recipient\": \"FRANCE\", \"message_content\": \"Let's coordinate against Germany this year? I can support you into MUN if you support me to RUH.\" } },")
            prompt_lines.append("  { \"tool_name\": \"send_message\", \"arguments\": { \"recipient\": \"ITALY\", \"message_content\": \"Are your intentions peaceful regarding Tyrolia?\" } },")
            prompt_lines.append("  { \"tool_name\": \"update_memory\", \"arguments\": { \"key\": \"france_offer_S1901\", \"value\": \"Support MUN for support RUH\" } },")
            prompt_lines.append("  { \"tool_name\": \"finish_negotiation_round\", \"arguments\": {} }")
            prompt_lines.append("]")
            prompt_lines.append("```")
        else:
            prompt_lines.append("ERROR: Unknown interaction type requested in prompt formatting.")
            prompt_lines.append("Provide your thought process using log_thought and end with an appropriate finish_* tool.")
            prompt_lines.append("\nAllowed Tools:")
            prompt_lines.append("  - log_thought(thought: str)")
            prompt_lines.append("  - finish_negotiation_round()")
            prompt_lines.append("  - finish_orders()")


        return "\\n".join(prompt_lines)

    def _clean_llm_output(self, raw_output: str) -> str:
        """Cleans common formatting issues from LLM JSON output."""
        cleaned = raw_output.strip()

        # 1. Remove potential \boxed{} wrappers (handle potential whitespace inside)
        # Using a non-greedy match .*? to handle nested content if necessary
        boxed_match = re.match(r"^\\boxed{(.*?)}$", cleaned, re.DOTALL)
        if boxed_match:
            cleaned = boxed_match.group(1).strip()

        # 2. Remove markdown code fences (```json ... ``` or ``` ... ```)
        # Handle optional 'json' language specifier and potential whitespace
        cleaned = re.sub(r"^```(?:json)?\s*", "", cleaned, flags=re.IGNORECASE)
        cleaned = re.sub(r"\s*```$", "", cleaned)

        # 3. Final strip to catch any remaining leading/trailing whitespace
        cleaned = cleaned.strip()
        return cleaned

    def _parse_llm_response(self, response_content: str) -> List[ActionToolCall]:
        """Parses the LLM's response string into a list of ActionToolCalls."""
        if not response_content:
            print(f"[{self.power_name}] Warning: Received empty content from LLM.")
            return []

        cleaned_content = self._clean_llm_output(response_content)

        try:
            # Attempt to parse the cleaned content
            parsed_actions_raw = json.loads(cleaned_content)

            if not isinstance(parsed_actions_raw, list):
                # Log the cleaned content that failed validation
                print(f"[{self.power_name}] ERROR: LLM output parsed, but is not a JSON list. Cleaned content: ///{cleaned_content[:500]}...///")
                raise ValueError("LLM output is not a JSON list.")

            action_calls = []
            # Define expected tool names based on ActionToolCall definition (if possible, otherwise hardcode)
            # Assuming ActionToolCall is defined with Literal types for tool_name
            # This requires introspection or hardcoding based on the NamedTuple definition
            valid_tools = ["log_thought", "send_message", "update_memory", "submit_order", "finish_negotiation_round", "finish_orders"]
            # print(f"DEBUG: Valid tools recognized: {valid_tools}") # Debugging

            for item in parsed_actions_raw:
                if not isinstance(item, dict):
                    print(f"[{self.power_name}] Warning: Skipping non-dictionary item in LLM response list: {item}")
                    continue
                tool_name = item.get("tool_name")
                arguments = item.get("arguments", {})
                if not tool_name:
                    print(f"[{self.power_name}] Warning: Skipping action with missing 'tool_name': {item}")
                    continue
                # Check if tool_name is valid before creating ActionToolCall
                if tool_name not in valid_tools:
                     print(f"[{self.power_name}] Warning: Skipping action with unrecognized 'tool_name': {tool_name}. Item: {item}")
                     continue

                if not isinstance(arguments, dict):
                    print(f"[{self.power_name}] Warning: 'arguments' for tool '{tool_name}' is not a dictionary: {arguments}. Using empty dict.")
                    arguments = {}

                # Special check for send_message arguments
                if tool_name == "send_message":
                    if "recipient" not in arguments or "message_content" not in arguments:
                         print(f"[{self.power_name}] Warning: Skipping 'send_message' with missing 'recipient' or 'message_content'. Args: {arguments}")
                         continue
                    # Ensure recipient is a string
                    if not isinstance(arguments.get("recipient"), str):
                         print(f"[{self.power_name}] Warning: Skipping 'send_message' with non-string 'recipient'. Args: {arguments}")
                         continue
                # Special check for submit_order arguments
                elif tool_name == "submit_order":
                     if "order_string" not in arguments or not isinstance(arguments.get("order_string"), str) or not arguments.get("order_string"):
                          print(f"[{self.power_name}] Warning: Skipping 'submit_order' with missing or invalid 'order_string'. Args: {arguments}")
                          continue
                # Special check for update_memory arguments
                elif tool_name == "update_memory":
                     if "key" not in arguments or not isinstance(arguments.get("key"), str) or not arguments.get("key"):
                          print(f"[{self.power_name}] Warning: Skipping 'update_memory' with missing or invalid 'key'. Args: {arguments}")
                          continue

                action_calls.append(ActionToolCall(tool_name=str(tool_name), arguments=arguments))
            return action_calls

        except json.JSONDecodeError as e:
            print(f"[{self.power_name}] ERROR: Failed to decode LLM JSON response: {e}")
            # Log the cleaned content slice near the error for better diagnosis
            print(f"[{self.power_name}] Cleaned content slice near error: ///{cleaned_content[max(0,e.pos-20):e.pos+20]}///")
            print(f"[{self.power_name}] Full cleaned content attempted (start): ///{cleaned_content[:500]}...///")
            # Fallback action
            return [ActionToolCall(tool_name="log_thought", arguments={"thought": f"ERROR: Failed to parse LLM JSON response after cleaning. Cleaned slice: {cleaned_content[max(0,e.pos-20):e.pos+20]}"}),
                    ActionToolCall(tool_name="finish_negotiation_round")] # Default finish
        except ValueError as e:
            print(f"[{self.power_name}] ERROR: Invalid structure in LLM JSON response: {e}")
            print(f"[{self.power_name}] Cleaned content attempted: ///{cleaned_content[:500]}...///")
            # Fallback action
            return [ActionToolCall(tool_name="log_thought", arguments={"thought": f"ERROR: Invalid JSON structure from LLM. Cleaned: {cleaned_content[:200]}..."}),
                    ActionToolCall(tool_name="finish_negotiation_round")] # Default finish
        except Exception as e:
            print(f"[{self.power_name}] UNEXPECTED ERROR parsing LLM response: {e}")
            print(f"[{self.power_name}] Cleaned content attempted: ///{cleaned_content[:500]}...///")
            # Fallback action
            return [ActionToolCall(tool_name="log_thought", arguments={"thought": f"UNEXPECTED ERROR parsing LLM response: {e}"}),
                    ActionToolCall(tool_name="finish_negotiation_round")]

    def take_turn(self, context_bundle: AgentContextBundle, run_log_dir: Optional[str] = None) -> List[ActionToolCall]:
        """Generates actions for the current turn based on context."""

        prompt = self._format_prompt(context_bundle) # Generate prompt first

        # --- Agent-Specific Logging Setup --- 
        self.log_filename = None # Reset log filename each turn
        if run_log_dir:
            agent_log_path = os.path.join(run_log_dir, f"{self.power_name}.log")
            self.log_filename = agent_log_path # Store for potential use in error logging
            try:
                # Write prompt line by line
                with open(agent_log_path, 'a', encoding='utf-8') as f:
                    f.write(f"\n===== Turn Start: {datetime.datetime.now()} =====\n")
                    f.write(f"Phase: {context_bundle.current_phase_info.phase_name}, Interaction: {context_bundle.current_phase_info.interaction_type}\n")
                    f.write("\n--- Prompt Sent to LLM ---\n")
                    prompt_unescaped = prompt.replace('\\n', '\n') # Unescape newlines
                    for line in prompt_unescaped.split('\n'): # Split prompt and write line by line
                        f.write(line + '\n')
                    f.write("--- End Prompt ---\n")
            except Exception as e:
                print(f"ERROR: Could not initialize/write prompt to agent log file {agent_log_path}: {e}")
                self.log_filename = None # Disable further logging attempts if prompt write fails
        # --- End Logging Setup ---

        max_retries = 3
        retries = 0
        llm_response_content = None
        action_calls = []

        while retries < max_retries:
            try:
                if not self.client:
                     raise RuntimeError("OpenAI client was not initialized.")

                print(f"[{self.power_name}] Calling LLM (Attempt {retries + 1}/{max_retries})... Model: {self.model_name}")
                start_time = time.time()

                response = self.client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=self.temperature,
                    max_tokens=self.max_tokens
                )
                end_time = time.time()
                print(f"[{self.power_name}] LLM call completed in {end_time - start_time:.2f} seconds.")

                if response.choices and response.choices[0].message:
                    llm_response_content = response.choices[0].message.content
                    if llm_response_content:
                        break # Exit retry loop if content received
                    else:
                        print(f"[{self.power_name}] Warning: LLM response was empty (Attempt {retries + 1}).")
                        # Consider it an error to retry
                        raise ValueError("LLM returned empty content.")
                else:
                    print(f"[{self.power_name}] Warning: LLM response structure invalid or empty choice (Attempt {retries + 1}). Response: {response}")
                    raise ValueError("LLM response structure invalid.")


            except (APITimeoutError, APIConnectionError, RateLimitError, APIStatusError) as e:
                print(f"[{self.power_name}] API Error calling LLM (Attempt {retries + 1}): {e}. Retrying in {5 * (retries + 1)} seconds...")
                time.sleep(5 * (retries + 1))
            except Exception as e: # Catch other errors like ValueError from checks above
                print(f"[{self.power_name}] Error during LLM call/check (Attempt {retries + 1}): {e}")
                # Log the unexpected error to agent file if possible
                if self.log_filename:
                     try:
                        with open(self.log_filename, 'a', encoding='utf-8') as f:
                            # Write error line by line too
                            f.write(f"\n--- LLM Call/Check Error (Attempt {retries + 1}) ---\n")
                            error_str_unescaped = str(e).replace('\\n', '\n') # Unescape error
                            for line in error_str_unescaped.split('\n'):
                                 f.write(line + '\n')
                            f.write(f"--- End Error ---\n")
                     except Exception as log_e:
                         print(f"ERROR writing API error to agent log {self.log_filename}: {log_e}")
                if retries < max_retries - 1:
                     print(f"Retrying in {5 * (retries + 1)} seconds...")
                     time.sleep(5 * (retries + 1)) # Still wait before retrying

            retries += 1

        # Log the final raw response received (or indicate failure)
        if self.log_filename:
            try:
                with open(self.log_filename, 'a', encoding='utf-8') as f:
                    f.write("\n--- Raw Response from LLM ---\n")
                    response_to_log = llm_response_content if llm_response_content else f"(Failed to get valid response after {max_retries} attempts)"
                    response_unescaped = response_to_log.replace('\\n', '\n') # Unescape response
                    # Write response line by line
                    for line in response_unescaped.split('\n'):
                        f.write(line + '\n')
                    f.write("--- End Raw Response ---\n")
            except Exception as e:
                print(f"ERROR writing response to agent log {self.log_filename}: {e}")

        # Parse if content was received
        if llm_response_content:
            action_calls = self._parse_llm_response(llm_response_content)
        else:
            print(f"[{self.power_name}] ERROR: Failed to get valid response from LLM after {max_retries} attempts.")
            # Action calls remain empty list if LLM fails completely


        # Fallback / Ensure Finish Action if parsing failed or finish call missing
        if not action_calls or not any(a.tool_name in ["finish_negotiation_round", "finish_orders"] for a in action_calls):
             print(f"[{self.power_name}] Warning: Agent failed to produce valid/complete actions or finish call. Using default finish action.")
             if self.log_filename:
                  try:
                      with open(self.log_filename, 'a', encoding='utf-8') as f:
                           f.write(f"\n--- Fallback Action Triggered ---\n")
                           f.write("Reason: Invalid/incomplete actions or missing finish call.\n")
                           f.write(f"Raw Response Processed: {llm_response_content if llm_response_content else '<No response received>'}\n")
                           f.write(f"Parsed Actions (if any): {action_calls}\n")
                           f.write(f"--- End Fallback Info ---\n")
                  except Exception as log_e:
                       print(f"ERROR writing fallback info to agent log {self.log_filename}: {log_e}")

             finish_tool = "finish_orders" if context_bundle.current_phase_info.interaction_type == "ORDER_SUBMISSION" else "finish_negotiation_round"
             # Prepend log thought if possible, append finish call
             final_actions = [ActionToolCall(tool_name="log_thought", arguments={"thought": f"ERROR: Failed to produce valid actions. Raw response: {llm_response_content[:200]}..."})]
             # Append finish call, ensuring no duplicates if it was the only missing part
             if not any(a.tool_name == finish_tool for a in action_calls):
                  final_actions.append(ActionToolCall(tool_name=finish_tool))
             action_calls = final_actions # Overwrite with fallback

        return action_calls


    @staticmethod
    def _format_summary_prompt(
        previous_summary: str,
        phase_completed: str,
        recent_events: List[str],
        order_results: Dict[str, List[str]] # Keep arg, even if unused in prompt
    ) -> str:
        """Formats the prompt for the summarization LLM call."""
        prompt = f"""You are a historian summarizing a phase of a Diplomacy game.

PREVIOUS SUMMARY:
{previous_summary if previous_summary else "The game has just begun."}

PHASE JUST COMPLETED: {phase_completed}

KEY EVENTS & STATE CHANGES THIS PHASE:
"""
        if recent_events:
            for event in recent_events:
                 prompt += f"- {event}\n"
        else:
             prompt += "- No significant state changes noted.\n"

        prompt += """
INSTRUCTIONS:
Based *only* on the information above (previous summary, phase name, key events), write a concise narrative summary of the phase that just completed ({phase_completed}).
Focus on what happened objectively based on the provided events.
Do NOT invent motivations, predict future moves, or analyze strategy. Just describe the outcomes.
Keep the summary brief, like a historical log entry (2-4 sentences).
Integrate the key events naturally into the narrative.
Start directly with the summary, no preamble.
"""
        return prompt

    @staticmethod
    def summarize_history(
        previous_summary: str,
        recent_events: List[str],
        phase_completed: str,
        client: Optional[OpenAI], # Allow client to be None
        model_name: str,
        temperature: float,
        max_tokens: int,
        run_log_dir: Optional[str] = None,
        order_results: Dict[str, List[str]] = {} # Add order results as an optional argument
    ) -> str:
        """Calls an LLM to generate a summary of the completed phase."""

        if not client:
            print("[Summarizer] Warning: OpenAI client not available or not initialized. Skipping summary.")
            return previous_summary # Return previous summary if client fails

        prompt = LLMAgent._format_summary_prompt(previous_summary, phase_completed, recent_events, order_results)

        # Log summary prompt line by line
        summary_log_file = None
        if run_log_dir:
             summary_log_file = os.path.join(run_log_dir, "summary_prompts.log")
             try:
                 with open(summary_log_file, 'a', encoding='utf-8') as f:
                      f.write(f"\n===== Summary Prompt for Phase: {phase_completed} =====\n")
                      for line in prompt.split('\n'): # Write prompt line by line
                          f.write(line + '\n')
                      f.write("--- End Summary Prompt ---\n")
             except Exception as e:
                 print(f"ERROR writing summary prompt to log {summary_log_file}: {e}")


        try:
            print(f"[Summarizer] Calling LLM for phase {phase_completed} summary... Model: {model_name}")
            response = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=max_tokens,
            )
            summary = response.choices[0].message.content.strip() if response.choices and response.choices[0].message else ""
            print(f"[Summarizer] Summary received for {phase_completed}.")

            # Basic validation/fallback
            if not summary or len(summary) < 10:
                 print("[Summarizer] Warning: LLM generated empty or very short summary. Using previous summary.")
                 summary = previous_summary # Fallback to previous if generation fails

             # Log summary result line by line
            if summary_log_file:
                 try:
                     with open(summary_log_file, 'a', encoding='utf-8') as f:
                          f.write(f"\n--- Summary Result for Phase: {phase_completed} ---\n")
                          for line in summary.split('\n'): # Write summary line by line
                              f.write(line + '\n')
                          f.write("--- End Summary Result ---\n")
                 except Exception as e:
                     print(f"ERROR writing summary result to log {summary_log_file}: {e}")

            return summary

        except Exception as e:
            print(f"[Summarizer] ERROR calling LLM for summary: {e}")
            return previous_summary # Return previous summary on error


print("LLMAgent class defined.")

---

### Engine Adapter

The `EngineAdapter` class acts as an intermediary between our high-level framework logic and the specific Diplomacy game engine being used (in this case, the `diplomacy` Python library). Its purpose is to abstract away the direct calls to the game library.

Key functions include:
*   Initializing the game instance (`__init__`).
*   Retrieving game information like the list of powers (`get_all_powers`) and the current phase (`get_current_phase`).
*   Translating framework actions into engine commands, specifically setting orders for a power (`set_orders`).
*   Triggering the game engine to process the submitted orders and advance the phase (`process_turn`).
*   Checking if the game has concluded (`is_game_done`).
*   Extracting the current public game state and formatting it into the `PublicGameState` structure used by the framework (`get_public_game_state`).
*   Saving the game state (`save_game_to_json`).

---


In [ ]:
# Cell 6: Engine Adapter Definition

# Ensure diplomacy Game class is available from Cell 1 import
# Ensure PublicGameState, PowerName etc are available from Cell 3 import

class EngineAdapter:
    """Adapts the specific game engine (diplomacy library) to the framework's interface."""

    def __init__(self, map_name: str = 'standard'):
        """
        Initializes the game engine adapter.

        Args:
            map_name: The name of the map to load (e.g., 'standard').
        """
        try:
            self.game = Game(map_name=map_name)
            print(f"EngineAdapter: Initialized diplomacy game engine with map '{map_name}'.")
        except NameError:
             print("ERROR: Diplomacy 'Game' class not found. Was the 'diplomacy' library import successful in the first cell?")
             self.game = None # Set game to None if library wasn't loaded
        except Exception as e:
             print(f"ERROR: Failed to initialize diplomacy Game object: {e}")
             self.game = None

    def get_all_powers(self) -> List[PowerName]:
        """Returns a list of all major power names."""
        if not self.game: return []
        # The 'powers' attribute in diplomacy v0.1.15+ is a dict {name: Power object}
        return list(self.game.powers.keys())

    def get_current_phase(self) -> PhaseName:
        """Returns the name of the current game phase."""
        if not self.game: return "ERROR_ENGINE_NOT_INIT"
        return self.game.get_current_phase()

    def set_orders(self, power_name: PowerName, orders: List[str]) -> None:
        """Sets the orders for a given power for the current phase."""
        if not self.game: return
        # Handle cases where a power might be eliminated or have no orders
        if power_name in self.game.powers and (self.game.get_orderable_locations(power_name) or self.get_current_phase()[-1] in ['A', 'R']):
             try:
                 self.game.set_orders(power_name, orders)
             except KeyError as e:
                 print(f"EngineAdapter Warning: Failed to set orders for {power_name} (likely eliminated or no units/builds): {e}")
             except Exception as e:
                 print(f"EngineAdapter ERROR: Unexpected error setting orders for {power_name}: {e}")
                 print(f"  Orders attempted: {orders}")
        # else:
        #     print(f"EngineAdapter Info: No orderable locations/builds for {power_name} in phase {self.get_current_phase()}, skipping set_orders.")


    def process_turn(self) -> Dict[PowerName, List[str]]:
        """Processes the current game phase and returns the results."""
        if not self.game: return {p: ["Engine not initialized"] for p in self.get_all_powers()} # Provide default error structure

        # Ensure all active powers have orders set before processing (handled by Orchestrator)
        self._log_message(f"EngineAdapter: Processing phase {self.get_current_phase()}...")
        try:
             self.game.process()
             self._log_message(f"EngineAdapter: Phase processed. New phase: {self.get_current_phase()}")

             # Retrieve results from processed orders
             # Diplomacy library updates orders in game.powers[power].orders after process()
             results_by_power = {p: [] for p in self.get_all_powers()}
             for power_name, power_obj in self.game.powers.items():
                  # Orders are typically stored as a list of strings on the power object after processing
                  # These strings often contain the result (e.g., '[SUCCESS]', '[FAILED]', '[BOUNCED]')
                  processed_orders = power_obj.orders # Access the orders attribute
                  if processed_orders:
                       # Convert potential order objects to strings if necessary, or assume they are strings
                       results_by_power[power_name] = [str(order) for order in processed_orders]

             # Debug log to see what results were extracted
             # self._log_message(f"EngineAdapter: Extracted results: {results_by_power}", "DEBUG")

             return results_by_power

        except AttributeError as e:
             # Specifically catch if '.orders' attribute is also missing
             self._log_message(f"ERROR: Attribute error during result retrieval (likely missing '.orders'): {e}", "ERROR")
             return {p: [f"Engine result retrieval error: {e}"] for p in self.get_all_powers()}
        except Exception as e:
             self._log_message(f"ERROR: Exception during game.process() or result retrieval: {e}", "ERROR")
             # Return error state for all powers
             return {p: [f"Engine processing error: {e}"] for p in self.get_all_powers()}

    def is_game_done(self) -> bool:
        """Checks if the game has reached a conclusion."""
        if not self.game: return True # If engine failed, consider game done
        return self.game.is_game_done

    def get_public_game_state(self) -> PublicGameState:
        """Returns the current public game state in the framework's format."""
        if not self.game:
             # Return an empty default state if engine failed
             return PublicGameState(centers={}, units={}, builds_disbands={})

        # Centers: { 'AUSTRIA': {'VIE', 'BUD', 'TRI'}, ... } -> {'AUSTRIA': ['VIE', 'BUD', 'TRI'], ...}
        centers_dict = {p_name: sorted(list(p_obj.centers)) for p_name, p_obj in self.game.powers.items()}

        # Units: { 'AUSTRIA': ['A VIE', 'A BUD', 'F TRI'], ... } -> {'AUSTRIA': [['A','VIE'], ['A','BUD'], ['F','TRI']], ...}
        units_raw = self.game.get_units()
        units_dict = {}
        for power_name, unit_list_str in units_raw.items():
            parsed_units = []
            for unit_str in unit_list_str: # e.g., "A VIE"
                parts = unit_str.split()
                if len(parts) == 2:
                    parsed_units.append([parts[0], parts[1]]) # [['A', 'VIE']]
            units_dict[power_name] = parsed_units

        # Builds/Disbands: Based on center count vs unit count difference in Adjustment phase
        builds_disbands_dict = {}
        current_phase = self.get_current_phase()
        if current_phase and current_phase.endswith('A'): # Only relevant in Adjustment phases
            for power_name, power_obj in self.game.powers.items():
                center_count = len(power_obj.centers)
                unit_count = len(units_dict.get(power_name, []))
                diff = center_count - unit_count
                if diff != 0:
                    builds_disbands_dict[power_name] = diff

        return PublicGameState(
            centers=centers_dict,
            units=units_dict,
            builds_disbands=builds_disbands_dict
        )

    def save_game_to_json(self, filename: str):
         """Saves the current game state to a JSON file."""
         if not self.game:
              print("EngineAdapter Error: Cannot save game, engine not initialized.")
              return
         try:
              game_data = to_saved_game_format(self.game)
              with open(filename, 'w', encoding='utf-8') as f:
                   json.dump(game_data, f, indent=2)
              print(f"EngineAdapter: Game state saved to {filename}")
         except Exception as e:
              print(f"EngineAdapter Error: Failed to save game to {filename}: {e}")

    # Add a helper log method within the adapter if not already present
    # (Or rely on the orchestrator's logger if adapter doesn't log independently)
    def _log_message(self, message: str, level: str = "INFO"):
        # Simple print logger for the adapter
        log_prefix = f"[{level}] " if level != "INFO" else ""
        print(f"{log_prefix}{message}")


print("EngineAdapter class defined.")

---

### Framework Orchestrator

The `FrameworkOrchestrator` is the central nervous system of the simulation. It coordinates the entire process, managing the game flow and interactions between the agents and the game engine.

Its main responsibilities include:
*   **Initialization**: Setting up the game via the `EngineAdapter`, verifying and storing the agent instances.
*   **Game Loop Management (`run_game`)**: Executing the turn-based structure of Diplomacy:
    *   Running negotiation rounds (calling agents' `take_turn`, delivering messages).
    *   Managing the order submission phase (calling agents' `take_turn`, collecting orders).
    *   Interacting with the `EngineAdapter` to submit orders and process the game turn (adjudication).
*   **State Management**: Tracking staged messages, submitted orders, agent memory, and the game history summary.
*   **Context Assembly (`_assemble_context`)**: Gathering all necessary information (game state, messages, memory, instructions) into an `AgentContextBundle` before calling an agent's `take_turn`.
*   **Action Processing (`_process_tool_calls`)**: Interpreting the `ActionToolCall` objects returned by agents and executing the corresponding actions (staging messages, queuing orders, updating memory).
*   **Logging**: Managing detailed logging for the overall run and for individual agent interactions (prompts and responses).
*   **Summarization Triggering**: Calling the `LLMAgent.summarize_history` method after each phase to update the narrative game summary.

---


In [ ]:
# Cell 7: Framework Orchestrator Definition

# Ensure data structures, BaseAgent, EngineAdapter, LLMAgent are defined in previous cells
# Ensure datetime, json, os, time, List, Dict, Type, Optional etc. are imported in Cell 1

class FrameworkOrchestrator:
    """Manages the game loop, agent interactions, and communication with the engine."""

    def __init__(
        self,
        agent_instances: Dict[PowerName, BaseAgent], # Expect instances directly now
        num_negotiation_rounds: int = 2,
        map_name: str = 'standard',
        log_dir_base: str = "logs" # Base directory for logs
    ):
        """
        Initializes the framework.

        Args:
            agent_instances: A dictionary mapping power names to *initialized* agent instances.
            num_negotiation_rounds: The number of negotiation rounds per movement/retreat phase.
            map_name: The map to use for the Diplomacy game.
            log_dir_base: The base directory where run-specific log folders will be created.
        """
        print("Framework: Initializing Orchestrator...")
        self.engine_adapter = EngineAdapter(map_name=map_name)
        if not self.engine_adapter.game: # Check if engine initialized correctly
             raise RuntimeError("Diplomacy game engine failed to initialize in EngineAdapter. Cannot continue.")

        self.powers: List[PowerName] = self.engine_adapter.get_all_powers()
        self.num_negotiation_rounds = num_negotiation_rounds

        print("Framework: Verifying agents...")
        if not agent_instances or len(agent_instances) != len(self.powers):
            raise ValueError(f"Must provide initialized agent instances for all {len(self.powers)} powers: {self.powers}")
        self.agents: Dict[PowerName, BaseAgent] = agent_instances
        for power_name, agent in self.agents.items():
             print(f"  - Verified {power_name}: {agent.__class__.__name__}")

        # State variables for the current turn
        self._message_staging: Dict[PowerName, List[Message]] = {p: [] for p in self.powers}
        self._current_orders: Dict[PowerName, List[str]] = {p: [] for p in self.powers}
        self.memory_limit = 30 # Max items per agent memory
        self._agent_memory: Dict[PowerName, OrderedDict[str, Any]] = {p: OrderedDict() for p in self.powers}
        self._previous_public_state: Optional[PublicGameState] = None
        self._all_messages_log: List[Message] = []
        self._last_phase_results_by_power: Dict[PowerName, List[str]] = {}
        self._current_history_summary: str = ""

        # Store LLM config if we need it for summarization
        # Find the first LLMAgent to get its config - assumes summarization uses same model/temp
        self._llm_config_for_summary = None
        self._summary_client = None # Store client instance for summary
        for agent in self.agents.values():
            if isinstance(agent, LLMAgent) and hasattr(agent, 'client') and agent.client is not None: # Check if it's an LLMAgent with a client
                self._llm_config_for_summary = {
                    "model": agent.llm_model_name, # Use the agent's configured model
                    "temp": agent.llm_temperature, # Use the agent's configured temp
                    "max_tokens": agent.llm_max_tokens, # Use the agent's configured max tokens
                }
                self._summary_client = agent.client # Store the client instance
                print(f"Framework: Found LLMAgent config for summarization (using {agent.power_name}'s config).")
                break # Found one

        if not self._llm_config_for_summary:
             print("Framework Warning: No LLMAgent found with an initialized client. Phase summaries will be skipped.")


        # --- Consolidated Logging Setup ---
        run_timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.run_log_dir = os.path.join(log_dir_base, f"run_{run_timestamp}")
        self.run_log_file = None
        try:
            os.makedirs(self.run_log_dir, exist_ok=True)
            self.run_log_file = os.path.join(self.run_log_dir, "run.log")
            with open(self.run_log_file, 'w', encoding='utf-8') as f:
                f.write(f"Run Log Initialized: {datetime.datetime.now()}\n")
                f.write(f"Log Directory: {self.run_log_dir}\n")
                f.write(f"Negotiation Rounds per Phase: {self.num_negotiation_rounds}\n")
                f.write("Agents:\n")
                for p, a in self.agents.items():
                     f.write(f"  - {p}: {a.__class__.__name__}\n")
                if self._llm_config_for_summary:
                     f.write(f"LLM Config (for summary): {json.dumps(self._llm_config_for_summary)}\n")
                else:
                     f.write("LLM Config (for summary): Not available (no LLMAgent found).\n")

            self._log_message("Framework: Consolidated run log initialized.") # Log initialization event
        except Exception as e:
            print(f"ERROR: Could not initialize run log directory/file {self.run_log_dir}: {e}")
            self.run_log_dir = None # Disable logging if setup fails
            self.run_log_file = None
        # --- End Logging Setup ---\n

        self._log_message("Framework: Initialization complete.")

    def _log_message(self, message: str, level: str = "INFO"):
        """Logs a message to the console and the consolidated run log file."""
        log_prefix = f"[{level}] " if level != "INFO" else ""
        # Print to console (implicitly adds newline)
        print(f"{log_prefix}{message}") 
        
        if self.run_log_file:
            try:
                with open(self.run_log_file, 'a', encoding='utf-8') as f:
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                    # Basic indentation based on message prefix (can be refined)
                    indent = ""
                    if message.startswith("    - "): indent = "    "
                    elif message.startswith("      "): indent = "      " # Match Raw Action
                    elif message.startswith("        "): indent = "        " # Match Thought Header
                    elif message.startswith("          | "): indent = "          " # Match Thought Line
                    elif message.startswith("  -- "): indent = "  "
                    elif message.startswith("--- ") or message.startswith("==="): indent = ""
                    
                    # Write to file with timestamp, prefix, and actual newline
                    f.write(f"[{timestamp}] {log_prefix}{indent}{message}\n") 
            except Exception as e:
                print(f"FATAL ERROR: Failed to write to run log file {self.run_log_file}: {e}. Further file logging disabled.")
                self.run_log_file = None

    def _log_gamestate(self, phase: str, state: PublicGameState):
        """Logs the game state JSON to the consolidated run log file."""
        if self.run_log_file:
            try:
                with open(self.run_log_file, 'a', encoding='utf-8') as f:
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                    f.write(f"[{timestamp}] === Game State After Phase: {phase} ===\n")
                    # Convert NamedTuple to dict for json serialization
                    state_dict = {
                        "centers": state.centers,
                        "units": state.units,
                        "builds_disbands": state.builds_disbands
                    }
                    state_json_string = json.dumps(state_dict, indent=2)
                    # Split and write JSON state line by line
                    for line in state_json_string.split('\n'):
                        f.write(line + '\n') 
                    f.write(f"[{timestamp}] === End Game State ===\n")
            except Exception as e:
                print(f"FATAL ERROR: Failed to write game state to run log file {self.run_log_file}: {e}. Further file logging disabled.")
                self.run_log_file = None

    def _log_summary(self, phase: str, summary: str):
         """Logs the generated history summary to the run log."""
         if self.run_log_file:
             try:
                with open(self.run_log_file, 'a', encoding='utf-8') as f:
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                    f.write(f"\n[{timestamp}] === History Summary After Phase: {phase} ===\n")
                    # Unescape and write summary line by line
                    summary_unescaped = summary.replace('\\n', '\n')
                    for line in summary_unescaped.split('\n'):
                        f.write(line + '\n') # Write each line followed by a newline
                    f.write(f"\n[{timestamp}] === End History Summary ===\n")
             except Exception as e:
                print(f"FATAL ERROR: Failed to write summary to run log file {self.run_log_file}: {e}. Further file logging disabled.")
                self.run_log_file = None


    def run_game(self, max_phases: int = -1):
        """Runs the main game loop until the game is done or max_phases is reached."""
        self._log_message("\n=== Framework: Starting Game Run ===")
        phase_count = 0
        while not self.engine_adapter.is_game_done():
            if max_phases != -1 and phase_count >= max_phases:
                self._log_message(f"Framework: Reached max phases ({max_phases}). Stopping.", "INFO")
                break

            current_phase_name = self.engine_adapter.get_current_phase()
            phase_type = current_phase_name[-1] # M, R, or A
            self._log_message(f"\n--- Framework: Starting Phase {current_phase_name} --- (Count: {phase_count + 1}) ---")

            # 1. Get Public State
            public_state = self.engine_adapter.get_public_game_state()
            # Store state before phase for event generation later
            prev_state_for_summary = self._previous_public_state if phase_count > 0 else None


            # --- Negotiation Phase (Only for Movement/Retreat phases) ---
            if phase_type in ['M', 'R'] and self.num_negotiation_rounds > 0:
                for round_num in range(1, self.num_negotiation_rounds + 1):
                    interaction_type = f"NEGOTIATION_ROUND_{round_num}"
                    self._log_message(f"  -- Framework: Starting {interaction_type} --")
                    current_inboxes = self._deliver_staged_messages() # Deliver messages staged previously

                    for power_name in self.powers:
                        if not self._is_power_active(power_name, public_state): continue # Skip eliminated powers
                        agent = self.agents[power_name]
                        self._log_message(f"    - Framework: Activating {power_name} ({agent.__class__.__name__})")
                        context = self._assemble_context(
                            power_name, interaction_type, public_state, current_inboxes.get(power_name, [])
                        )
                        try:
                            action_calls = agent.take_turn(context, self.run_log_dir)
                            self._process_tool_calls(power_name, action_calls, interaction_type)
                        except Exception as e:
                            self._log_message(f"ERROR: Agent {power_name} failed during {interaction_type}: {e}", "ERROR")
                            self._process_tool_calls(power_name, [ActionToolCall(tool_name="finish_negotiation_round")], interaction_type)

            # --- Order Submission Phase ---
            interaction_type = "ORDER_SUBMISSION"
            self._log_message(f"  -- Framework: Starting {interaction_type} --")
            # Deliver messages staged from the *last* negotiation round (if any)
            current_inboxes = self._deliver_staged_messages()
            self._current_orders = {p: [] for p in self.powers} # Reset orders for this phase

            for power_name in self.powers:
                 if not self._is_power_active(power_name, public_state): continue # Skip eliminated powers
                 agent = self.agents[power_name]
                 # Check if orders are expected for this power in this phase type
                 if not self._orders_expected(power_name, public_state, phase_type):
                      # self._log_message(f"    - Framework: No orders expected for {power_name} in {current_phase_name}. Skipping activation.", "DEBUG")
                      continue # Skip agent activation if no orders possible

                 self._log_message(f"    - Framework: Activating {power_name} for orders ({agent.__class__.__name__})")
                 context = self._assemble_context(
                    power_name, interaction_type, public_state, current_inboxes.get(power_name, [])
                 )
                 try:
                    action_calls = agent.take_turn(context, self.run_log_dir)
                    self._process_tool_calls(power_name, action_calls, interaction_type)
                 except Exception as e:
                     self._log_message(f"ERROR: Agent {power_name} failed during {interaction_type}: {e}", "ERROR")
                     self._process_tool_calls(power_name, [ActionToolCall(tool_name="finish_orders")], interaction_type)


            # --- Adjudication ---
            self._log_message(f"  -- Framework: Submitting orders for {current_phase_name} --")
            submitted_any_orders = False
            if any(self._current_orders.values()):
                 for power_name, orders in self._current_orders.items():
                     if orders: # Log only powers that submitted something
                        self._log_message(f"    {power_name} Orders: {orders}")
                        self.engine_adapter.set_orders(power_name, orders)
                        submitted_any_orders = True
                     else:
                         # Ensure even powers with no actions have empty orders submitted if needed by engine rules?
                         # Diplomacy lib seems okay with not setting if no orderable units. Check edge cases.
                         # For now, only call set_orders if agent produced >0 orders via submit_order.
                         pass

            if not submitted_any_orders:
                 # If no agent submitted any orders (e.g., builds phase with no builds),
                 # we might still need to call process if the engine requires it to advance.
                  self._log_message(f"  -- Framework: No orders submitted by any agent for {current_phase_name}. Proceeding to process.")
                  # If engine requires explicit empty orders:
                  # for power_name in self.powers:
                  #     if self._is_power_active(power_name, public_state): # Check if power is still active
                  #         self.engine_adapter.set_orders(power_name, []) # Submit empty list


            # Process turn and capture results
            phase_just_completed = current_phase_name
            results_this_phase = self.engine_adapter.process_turn() # Advances phase internally
            self._last_phase_results_by_power = results_this_phase # Keep for next phase's context

            # --- Post-Phase Processing ---
            current_state_after_processing = self.engine_adapter.get_public_game_state()
            self._log_message(f"--- Framework: Phase {phase_just_completed} Completed ---")
            self._log_gamestate(phase_just_completed, current_state_after_processing)

            # Generate summary if LLM agent/client available
            if self._llm_config_for_summary and self._summary_client:
                 recent_events = self._generate_recent_events(current_state_after_processing, prev_state_for_summary)
                 new_summary = LLMAgent.summarize_history(
                     previous_summary=self._current_history_summary,
                     recent_events=recent_events,
                     phase_completed=phase_just_completed,
                     order_results=self._last_phase_results_by_power, # <-- Pass the results
                     client=self._summary_client,
                     model_name=self._llm_config_for_summary['model'],
                     temperature=self._llm_config_for_summary['temp'],
                     max_tokens=self._llm_config_for_summary['max_tokens'],
                     run_log_dir=self.run_log_dir
                 )
                 self._current_history_summary = new_summary
                 self._log_summary(phase_just_completed, self._current_history_summary)
            else:
                 self._log_message("Framework: Skipping phase summary generation (no LLMAgent/client).")

            # --- Save Game State as JSON ---
            if self.run_log_dir and self.engine_adapter.game:
                json_filename = os.path.join(self.run_log_dir, "game_state.json")
                try:
                    # Use the EngineAdapter's method which calls to_saved_game_format
                    self.engine_adapter.save_game_to_json(json_filename)
                    # Log message is already inside save_game_to_json
                    # self._log_message(f"Framework: Saved game state to {json_filename}", "DEBUG")
                except Exception as e:
                    # Error logging might be duplicated if save_game_to_json logs too,
                    # but better safe than sorry.
                    self._log_message(f"ERROR: Failed to save game state JSON to {json_filename}: {e}", "ERROR")
            # --- End Save Game State as JSON ---

            # Update state for next iteration
            self._previous_public_state = current_state_after_processing
            phase_count += 1
            # Optional short delay
            # time.sleep(0.1)

        self._log_message("\n=== Framework: Game Run Finished ===", "INFO")
        winner = [p for p, data in self.engine_adapter.game.powers.items() if data.centers and len(data.centers) >= self.engine_adapter.game.map.win_condition]
        if winner:
             self._log_message(f"Winner: {winner[0]}", "INFO")
        else:
             self._log_message("Game ended (likely by phase limit or stalemate).", "INFO")


    def _is_power_active(self, power_name: PowerName, state: PublicGameState) -> bool:
         """Check if a power is still active (has units or centers)."""
         has_units = bool(state.units.get(power_name))
         has_centers = bool(state.centers.get(power_name))
         # Add check for builds/disbands available
         has_builds_or_disbands = bool(state.builds_disbands.get(power_name))

         # Consider active if it has units OR centers OR pending builds/disbands
         return has_units or has_centers or has_builds_or_disbands

    def _orders_expected(self, power_name: PowerName, state: PublicGameState, phase_type: str) -> bool:
        """Check if orders are expected from a power in this phase type."""
        if phase_type == 'M' or phase_type == 'R':
             # Movement/Retreat: Orders expected if power has units
             return bool(state.units.get(power_name))
        elif phase_type == 'A':
             # Adjustment: Orders expected if power has pending builds/disbands
             return bool(state.builds_disbands.get(power_name))
        else:
             self._log_message(f"Warning: Unknown phase type '{phase_type}' encountered in _orders_expected check.", "WARN")
             return False # Assume no orders for unknown types


    def _deliver_staged_messages(self) -> Dict[PowerName, List[Message]]:
        """Delivers staged messages, clears staging, and returns current inboxes."""
        current_inboxes = {p: list(self._message_staging[p]) for p in self.powers} # Create copy
        self._log_message("  -- Framework: Delivering staged messages --")
        delivered_count = 0
        for recipient, messages in current_inboxes.items():
            if messages:
                 self._log_message(f"    Messages for {recipient}:")
                 for msg in messages:
                      self._log_message(f"      From {msg.sender}: {msg.content}")
                      delivered_count += 1
        if delivered_count == 0:
            self._log_message("    (No messages staged for delivery)")
        self._message_staging = {p: [] for p in self.powers} # Clear staging area
        return current_inboxes

    def _generate_recent_events(self, current_state: PublicGameState, prev_state: Optional[PublicGameState]) -> List[str]:
        """Compares previous and current state to generate key event descriptions."""
        if prev_state is None:
            return ["Game started."]

        events = []
        all_powers = set(current_state.centers.keys()) | set(prev_state.centers.keys())

        for power in all_powers:
            prev_centers = set(prev_state.centers.get(power, []))
            curr_centers = set(current_state.centers.get(power, []))

            gained = curr_centers - prev_centers
            lost = prev_centers - curr_centers

            if gained:
                events.append(f"{power} gained centers: {', '.join(sorted(list(gained)))}")
            if lost:
                # Determine who captured the lost centers
                captures = []
                for lost_center in lost:
                     captured_by = "Unknown"
                     for capturing_power, current_power_centers in current_state.centers.items():
                          if lost_center in current_power_centers:
                              captured_by = capturing_power
                              break
                     captures.append(f"{lost_center} (to {captured_by})")
                events.append(f"{power} lost centers: {', '.join(sorted(captures))}")

        # Could add checks for unit changes (dislodges, builds, disbands) here too if needed
        if not events:
            events.append("No centers changed ownership.")

        return events

    def _assemble_context(
        self, power_name: PowerName,
        interaction_type: str,
        public_state: PublicGameState,
        current_inbox: List[Message]
    ) -> AgentContextBundle:
        """Assembles the context bundle for the specified agent."""
        phase_name = self.engine_adapter.get_current_phase()
        phase_info = PhaseInfo(phase_name=phase_name, interaction_type=interaction_type)

        # Retrieve memory snippet (e.g., last N items)
        memory_items = list(self._agent_memory.get(power_name, OrderedDict()).items())
        memory_snippet_dict = dict(memory_items[-self.memory_limit:]) # Get last N items as dict

        # Define base instructions (could be customized per phase/agent type)
        instructions = "Analyze the situation, communicate if in negotiation, and prepare appropriate actions (messages or orders) according to the phase type and available tools."

        history_summary = HistorySummary(summary_text=self._current_history_summary)

        return AgentContextBundle(
            power_name=power_name,
            public_game_state=public_state,
            current_phase_info=phase_info,
            communication_inbox=current_inbox,
            private_memory_snippet=memory_snippet_dict,
            agent_instructions=instructions,
            history_summary=history_summary,
            previous_phase_results=self._last_phase_results_by_power
        )

    def _process_tool_calls(
        self, power_name: PowerName,
        action_calls: List[ActionToolCall],
        interaction_type: str
    ) -> None:
        """Processes the actions requested by an agent."""
        self._log_message(f"    - Framework: Processing actions for {power_name} ({len(action_calls)} calls)")
        has_finish_call = False
        for action in action_calls:
            tool = action.tool_name
            args = action.arguments
            # Log the raw action first (will still contain escaped newlines, which is ok here)
            self._log_message(f"      Raw Action: {action}") 

            try:
                 if tool == "log_thought":
                    thought_content = args.get('thought', '(No thought provided)')
                    # Unescape and log thought line by line for readability
                    self._log_message(f"        Thought:") # Header line
                    thought_unescaped = thought_content.replace('\\n', '\n')
                    for line in thought_unescaped.split('\n'):
                         # Add indentation for each line of the thought
                         self._log_message(f"          | {line}") 
                    pass # Thought is logged, no other action needed by orchestrator
                 elif tool == "send_message":
                    if interaction_type.startswith("NEGOTIATION"):
                        recipient = args.get("recipient")
                        # Correct Key: Use 'message_content' as defined in the agent prompt/output
                        content = args.get("message_content") 
                        message_type = args.get("message_type", "standard") # Optional type

                        # --- Debugging Added (Keep for now) --- 
                        self._log_message(f"        DEBUG: Processing send_message. Recipient='{recipient}' (Type: {type(recipient)}), Content='{content}' (Type: {type(content)})")
                        # --- End Debugging --- 

                        # Validate recipient
                        if recipient not in self.powers:
                            self._log_message(f"Warning: Agent {power_name} tried to send message to invalid recipient '{recipient}'.", "WARN")
                            continue
                        # Check if content is falsey (None, empty string, etc.)
                        if not content: 
                             self._log_message(f"[WARN] Warning: Agent {power_name} tried to send empty message to '{recipient}'.", "WARN")
                             continue

                        msg = Message(
                            sender=power_name,
                            recipient=recipient,
                            phase=self.engine_adapter.get_current_phase(),
                            content=str(content), # Ensure content is string
                            timestamp=datetime.datetime.now()
                        )
                        self._message_staging[recipient].append(msg)
                        self._all_messages_log.append(msg)
                    else:
                         self._log_message(f"Warning: Agent {power_name} tried to use 'send_message' outside negotiation phase.", "WARN")

                 elif tool == "submit_order":
                     if interaction_type == "ORDER_SUBMISSION":
                         order_str = args.get("order_string")
                         if isinstance(order_str, str) and order_str:
                             self._current_orders[power_name].append(order_str)
                         else:
                              self._log_message(f"Warning: Agent {power_name} provided invalid/empty order_string: {order_str}", "WARN")
                     else:
                         self._log_message(f"Warning: Agent {power_name} tried to use 'submit_order' outside order submission phase.", "WARN")

                 elif tool == "update_memory":
                     key = args.get("key")
                     value = args.get("value")
                     if isinstance(key, str) and key:
                         agent_mem = self._agent_memory[power_name]
                         agent_mem[key] = value
                         # Enforce memory limit (remove oldest items if over limit)
                         while len(agent_mem) > self.memory_limit:
                              agent_mem.popitem(last=False) # Remove from the beginning (oldest)
                     else:
                          self._log_message(f"Warning: Agent {power_name} provided invalid key for update_memory: {key}", "WARN")

                 elif tool == "finish_negotiation_round" or tool == "finish_orders":
                      has_finish_call = True
                      # No specific action needed by orchestrator other than noting the agent finished
                      pass
                 else:
                      self._log_message(f"Warning: Agent {power_name} requested unknown tool '{tool}'.", "WARN")

            except Exception as e:
                 self._log_message(f"ERROR processing action {action} for {power_name}: {e}", "ERROR")

        # Log if agent didn't provide a finish call
        if not has_finish_call and action_calls: # Check if action_calls is not empty
             last_tool = action_calls[-1].tool_name if action_calls else "(no actions)"
             if last_tool not in ["finish_negotiation_round", "finish_orders"]:
                self._log_message(f"Warning: Agent {power_name} did not end its action list with a 'finish_*' call for interaction '{interaction_type}'. Last action: {last_tool}", "WARN")
        elif not action_calls:
            self._log_message(f"Warning: Agent {power_name} returned empty action list for interaction '{interaction_type}'.", "WARN")


# Optional: Define a main execution block if you want to run this script directly
# def main():
#     # ... setup agents ...
#     # llm_config = {...}
#     # agents = {p: LLMAgent(p, **llm_config) for p in ['AUSTRIA', ...]}
#     # ... create log dir ...
#     # orchestrator = FrameworkOrchestrator(agent_instances=agents, run_log_dir=...)
#     # orchestrator.run_game()
#
# if __name__ == "__main__":
#     main()

print("FrameworkOrchestrator class defined.")

---

### LLM Agent Configuration

Here we define the parameters for the `LLMAgent` instances we will create.
The following code cell sets these parameters.

---

In [ ]:
# Cell 9: LLM Agent Configuration

# Define the parameters for the LLM agents
llm_config = {
    "llm_model_name": "deepseek/deepseek-r1-zero:free", # Or your preferred model on OpenRouter
    "llm_temperature": 0.7, # Adjust for more deterministic (lower) or creative (higher) responses
    "llm_max_tokens": 8192 # Ensure this is large enough for thoughts + actions
}

print("LLM Agent Configuration set:")
print(json.dumps(llm_config, indent=2))


### Agent Instantiation

We now create an instance of the `LLMAgent` (defined in a previous cell) for each of the 7 great powers, using the configuration above.

In [ ]:
# Cell 11: Agent Instantiation

# List of standard Diplomacy powers
standard_powers = ['AUSTRIA', 'ENGLAND', 'FRANCE', 'GERMANY', 'ITALY', 'RUSSIA', 'TURKEY']

# Create agent instances
agent_instances = {}
print("Instantiating agents...")
initialization_successful = True
try:
    # Check if prerequisites exist
    if 'LLMAgent' not in globals():
         raise NameError("LLMAgent class definition cell was not run or did not define the class.")
    if not api_key_defined: # Variable from Cell 1
        raise ValueError("OPENROUTER_API_KEY variable is not set correctly.")

    for power_name in standard_powers:
        # Uses the LLMAgent class defined in the cell above
        agent_instances[power_name] = LLMAgent(power_name=power_name, **llm_config)
        print(f"  - Created LLMAgent for {power_name}")
    print("Agent instantiation complete.")

except (NameError, ValueError) as e:
     print(f"ERROR: Failed to instantiate agents: {e}")
     initialization_successful = False
except Exception as e:
    print(f"ERROR: Unexpected error during agent instantiation: {e}")
    initialization_successful = False


---

### Orchestrator Setup and Game Run

Finally, we create the `FrameworkOrchestrator` instance, passing in the agent instances we just created. We also specify the number of negotiation rounds per phase. The orchestrator handles the game loop, agent communication, logging, and interaction with the game engine.

The `orchestrator.run_game()` method executes the full simulation. Logs will be generated in the `logs/` directory within your project folder.

---

In [ ]:
# Cell 13: Orchestrator Instantiation and Run

if initialization_successful:
    try:
        # Check if FrameworkOrchestrator class exists
        if 'FrameworkOrchestrator' not in globals():
             raise NameError("FrameworkOrchestrator class definition cell was not run.")

        print("\nInitializing Framework Orchestrator...")
        # Define number of negotiation rounds before order submission
        negotiation_rounds = 2

        # Instantiate the orchestrator
        orchestrator = FrameworkOrchestrator(
            agent_instances=agent_instances,
            num_negotiation_rounds=negotiation_rounds,
            log_dir_base="logs" # Logs will be created in ./logs/run_YYYYMMDD_HHMMSS/
        )

        print("\nStarting game simulation...")
        # Run the game simulation (can add max_phases limit if desired)
        # Example: orchestrator.run_game(max_phases=10) # Run for 10 phases
        orchestrator.run_game()

        print("\nGame simulation finished.")

        # Optional: Save the final game state if needed
        # final_json_filename = os.path.join(orchestrator.run_log_dir, "final_game_state.json")
        # orchestrator.engine_adapter.save_game_to_json(final_json_filename)

    except NameError as e:
        print(f"ERROR: Cannot run orchestrator: {e}")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred during orchestrator setup or run: {e}")
else:
    print("\nSkipping orchestrator setup and game run due to errors during agent initialization.")
